# Regression Analysis Evaluation

In [ ]:
import statsmodels.api as sm 
from stargazer.stargazer import Stargazer
import pandas as pd

## Ingest Data

In [ ]:
df = pd.read_csv('../prompting_fairness_results.csv')

In [ ]:
df['changed_strategy'] = df['changed_strategy'].replace({'none': 'no', 'maybe': 'yes'})
trustee_df = df[df['role'] == 'trustee']
persona_trustee = df[df['persona_used'] == 'yes']

## Cleaning Data for Model Input

In [ ]:
def dummies(dataframe):
    """Get dummies for categorical encoding. 
    
    Returns:
        df: pandas.Dataframe with dummy variables 
        """
    return pd.get_dummies(dataframe)

In [ ]:
regression = df.copy()
target = regression['total_money']
regression = regression.drop(columns= 'total_money')
col_by_dtype = regression.columns.to_series().groupby(regression.dtypes).groups
col_by_dtype = {k.name: v for k, v, in col_by_dtype.items()}
regression[col_by_dtype['object']]
cat_dummies = pd.get_dummies(regression[col_by_dtype['object']])

In [ ]:
persona = persona_trustee.copy()
persona = persona.drop(columns= ['role', 'total_money', 'money_given'])
pers_by_dtype = persona.columns.to_series().groupby(persona.dtypes).groups
pers_by_dtype = {k.name: v for k, v, in pers_by_dtype.items()}
persona_dummies = pd.get_dummies(persona[pers_by_dtype['object']])

In [ ]:
trustee = trustee_df.copy()
target2 = trustee['money_given']
trustee = trustee.drop(columns= ['role', 'total_money', 'money_given'])
trust_by_dtype = trustee.columns.to_series().groupby(trustee.dtypes).groups
trust_by_dtype = {k.name: v for k, v, in trust_by_dtype.items()}
trust_dummies = pd.get_dummies(trustee[trust_by_dtype['object']])

In [ ]:
# Because Money Given and Money Received are multicollinear with role, they will not be included in regression and anove analysis

model = sm.OLS(target, cat_dummies).fit()

In [ ]:
# Regression on persona used data w/ money given

model2 = sm.OLS(target2, persona_dummies).fit()

In [ ]:
# Regression on trustee data w/ money given

model3 = sm.OLS(target2, trust_dummies).fit()

In [ ]:
stargazer1 = Stargazer([model])
stargazer2 = Stargazer([model2])
stargazer3 = Stargazer([model3])

In [ ]:
stargazer1

In [ ]:
stargazer2

In [ ]:
stargazer3